# Import packages

In [1]:
# imported packages
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import pickle
%matplotlib inline
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as plticker
import bisect
import copy
import json
import csv
import ast
# from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d
import time
import os
import scipy.integrate as integrate
import pandas as pd

# Lists of colors for plots

c0 = (0.76, 0.76, 0.76)
c1 = (1.00, 0.18, 0.33);
c2 = (1.00, 0.23, 0.19);
c3 = (1.00, 0.58, 0.00);
c4 = (1.00, 0.80, 0.00);
c5 = (0.30, 0.85, 0.39);
c6 = (0.35, 0.78, 0.98);
c7 = (0.20, 0.67, 0.86);
c8 = (0.00, 0.48, 1.00);
c9 = (0.35, 0.34, 0.84);
c10 = (0.00, 0.31, 0.57);
c11 = (0.12, 0.29, 0.69);
c12 = (0.17, 0.17, 0.42);
c13 = (1.00, 1.00, 1.00);
c14 = (0.77, 0.04, 0.00);

ios_colors=[c5, c1, c3, c7, c2, c6, c4, c9, c8, c10, c14, c12]

def to_rgb_frac(vect):
    return (float(vect[0])/255, float(vect[1])/255, float(vect[2])/255)

color_list=list(map(to_rgb_frac, [(166,206,227),(178,223,138),(51,160,44),(31,120,180)]))

# Lists of colors for plots

def to_rgb_frac(vect):
    return (float(vect[0])/255, float(vect[1])/255, float(vect[2])/255)

rb_div_5=list(map(to_rgb_frac, [(202,0,32),
(244,165,130),
(247,247,247),
(146,197,222),
(5,113,176)]))

qualitative_10=list(map(to_rgb_frac, [   
(99,99,99),
(178,223,138),
(51,160,44),
(251,154,153),
(227,26,28),
(253,191,111),
(255,127,0),
(202,178, 214),
(106,61,154),
(166,206,227),    
(31,120,180),  
(189,189,189),
]))

#define the colors from colorbrewer2
orange1 = '#feedde'
orange2 = '#fdbe85'
orange3 = '#fd8d3c'
orange4 = '#e6550d'
orange5 = '#a63603'
blue1 = '#eff3ff'
blue2 = '#bdd7e7'
blue3 = '#6baed6'
blue4 = '#3182bd'
blue5 = '#08519c'
green1 = '#edf8e9'
green2 = '#bae4b3'
green3 = '#74c476'
green4 = '#31a354'
green5 = '#006d2c'
grey1 = '#f7f7f7'
grey2 = '#cccccc'
grey3 = '#969696'
grey4 = '#636363'
grey5 = '#252525'
purple1 = '#f2f0f7'
purple2 = '#cbc9e2'
purple3 = '#9e9ac8'
purple4 = '#756bb1'
purple5 = '#54278f'
red1 = '#fee5d9'
red2 = '#fcae91'
red3 = '#fb6a4a'
red4 = '#de2d26'
red5 = '#a50f15'
yellow = '#ffffd4'


zeroth_mutant_colors=[grey4]
single_mutant_colors=[blue4]
double_mutant_colors=[orange3]
triple_mutant_colors=[green4]
quadruple_mutant_colors=[red5]

multiple_mutant_colors=[zeroth_mutant_colors, \
                        single_mutant_colors, \
                        double_mutant_colors, \
                        triple_mutant_colors, \
                        quadruple_mutant_colors]

# Data structure

Object that gets dynamically updated is a dictionary `clones`

The keys of this dictionary are unique integer identifiers the entry is another dictionary with keys:

+ `clone_size_trajectory`  - an array of length `T` (typically 1000) of clone size over time 
+ `current_clone_size` - integer of current clone size in population
+ `mutations`  - a dictionary of `{mut_id:fitness, }`
+ `parent` - integer ID of parent clone
+ `children` - integer IDs of all children clones
+ `fitness` - real number decribing net birth rate

# Functions for performing dynamics

In [4]:
def mutate_population(clones, last_mutation_dict, mut_rate, DFEs, t, dt, T):
    
    mutation_counter=last_mutation_dict['last_mutation']
    
    clones_copy=copy.deepcopy(clones)
    
    for clone_id, clone_entry in clones_copy.items():
        
        size=clone_entry['current_clone_size']
        mutations=clone_entry['mutations']
        number_muts_in_clone=len(mutations)  
        
        if number_muts_in_clone<4 and size>0:
            expected_children=size*mut_rate*dt
            number_of_children=np.random.poisson(expected_children)
#             DFE=DFEs[number_muts_in_clone]
        else:
            number_of_children=0
        
        for new_child in range(number_of_children):
            
            mutation_counter+=1
            child_id=mutation_counter
            if number_muts_in_clone==0:
                new_mutation_fitness=s1
            if number_muts_in_clone==1:
                new_mutation_fitness=s2
            if number_muts_in_clone==2:
                new_mutation_fitness=s3
            if number_muts_in_clone==3:
                new_mutation_fitness=s4
            
            clones[child_id]={}
            clones[child_id]['clone_size_trajectory']=np.zeros(T)
            clones[child_id]['clone_size_trajectory'][t]=1
            clones[child_id]['parent']=clone_id
            clones[child_id]['children']=[]
            clones[child_id]['current_clone_size']=1
            clones[child_id]['mutations']=copy.deepcopy(mutations)
            clones[child_id]['mutations'][mutation_counter]=new_mutation_fitness
            clones[clone_id]['children'].append(child_id)
            
            total_clone_fitness=0.0
            for m, f in clones[child_id]['mutations'].items():
                total_clone_fitness=total_clone_fitness+f
            clones[child_id]['fitness']=total_clone_fitness      
            
    clones=clones_copy
    last_mutation_dict['last_mutation']=mutation_counter
    
    
def mean_fitness(clones):
    total_population_size=0
    mean_fitness=0.0
    for clone_id, clone_entry in clones.items():
        total_population_size=total_population_size+clone_entry['current_clone_size']
    for clone_id, clone_entry in clones.items():
        clone_size=clone_entry['current_clone_size'] 
        clone_frequency=clone_size/total_population_size
        clone_fitness=clone_entry['fitness']
        mean_fitness=mean_fitness+clone_fitness*clone_frequency
        
    return mean_fitness, total_population_size

def select_population(clones, mean_fitness, t, dt, total_population_size, list_of_t1_t2_t3_t4_n4):
    
    for clone_id, clone_entry in clones.items():
        
        size=clone_entry['current_clone_size']
        fitness=clone_entry['fitness']
        mutations=clone_entry['mutations']
        lead=fitness-mean_fitness
        new_expected_size=size*np.exp(lead*dt)   
        
        if new_expected_size>0:
            new_size=np.random.poisson(new_expected_size)
        else:
            new_size=0
            
        if new_size>0:              ###      
            k=len(mutations)
            if k != 0:
                if mutations_acquired[k-1]==False:
#                     if k==3:
#                         print('first',k,'-th mutant occurred at time', t)
                    list_of_t1_t2_t3_t4_n4[k-1] =  t
                    mutations_acquired[k-1]=True
        clone_entry['current_clone_size']=new_size
        clone_entry['clone_size_trajectory'][t]=new_size
        
def aml_diagnosis_Poon(clones):

    total_population_size=0
    total_quadruple_size=0
    
    for clone_id, clone_entry in clones.items():
        total_population_size=total_population_size+clone_entry['current_clone_size']
    
    for clone_id, clone_entry in clones.items():
        
        mutations=clone_entry['mutations']
        
        if len(mutations)==4:
            total_quadruple_size=total_quadruple_size+clone_entry['current_clone_size']
            clone_entry['aml_clone']=True
        
        quandruple_mutant_frequency=total_quadruple_size/total_population_size
        
    if quandruple_mutant_frequency>1/total_population_size:  #         AML criterion = quadruple mutant>1 cell
        aml_dx=True
    else:
        aml_dx=False
    
    return aml_dx, quandruple_mutant_frequency


def population_size_trajectory(clones, T):
    
    total_population_size_traj=np.zeros(T)
    for clone_id, clone_entry in clones.items():
        total_population_size_traj=total_population_size_traj+clone_entry['clone_size_trajectory']

    return total_population_size_traj

def purge(clones):
    
    clones_to_be_purged=[]
    
    for clone_id, clone_entry in clones.items():
        
        size=clone_entry['current_clone_size']
        n_children=len(clone_entry['children'])
        max_historical_size=np.max(clone_entry['clone_size_trajectory'])
        
        if size==0 and n_children==0 and max_historical_size<10:
            clones_to_be_purged.append(clone_id)
            
    for c_id in clones_to_be_purged:
        del clones[c_id]
        
        
def variant_trajectories(clones, T):
    
    mutation_trajectories={}
    
    new_clones=copy.deepcopy(clones)
    
    pop_size_trajectory=population_size_trajectory(clones, T)
    
    for clone_id, clone_entry in new_clones.items():

        clone_size_trajectory=clone_entry['clone_size_trajectory']
        mutations=clone_entry['mutations']
        number_of_mutants_in_clone=len(mutations)

        for mutation_id, mutation_fitness in mutations.items():

            if mutation_id in mutation_trajectories:
                current_mutation_trajectory=mutation_trajectories[mutation_id]['mutation_size_trajectory']
                new_mutation_trajectory=current_mutation_trajectory+clone_size_trajectory/pop_size_trajectory
                mutation_trajectories[mutation_id]['mutation_size_trajectory']=new_mutation_trajectory
                current_mm_number=mutation_trajectories[mutation_id]['multiple_mutant_number']
                if number_of_mutants_in_clone<current_mm_number:
                    mutation_trajectories[mutation_id]['multiple_mutant_number']=number_of_mutants_in_clone
            else: #if mutation not in mutation_trajectories
                mutation_trajectories[mutation_id]={}
                mutation_trajectories[mutation_id]['mutation_size_trajectory']=clone_size_trajectory/pop_size_trajectory 
                mutation_trajectories[mutation_id]['mutation_fitness']=mutation_fitness
                mutation_trajectories[mutation_id]['multiple_mutant_number']=number_of_mutants_in_clone
                
    return mutation_trajectories
        
        
def plot_clone_trajectories(clones, T, dt, sim):
    
    plt.close('all')
    fig, ax1 = plt.subplots(1, 1, figsize = (20, 8))
    plt.subplots_adjust(hspace=0.3)

    total_clone_size_trajectory=np.zeros(T)
    for clone_id, clone_entries in clones.items():

        clone_size_traj=clone_entries['clone_size_trajectory']
        total_clone_size_trajectory=total_clone_size_trajectory+clone_size_traj

    for clone_id, clone_entries in clones.items():

        mutations=clone_entries['mutations']
        mm=len(mutations)
        clone_size_traj=clone_entries['clone_size_trajectory']
        clone_frequency_trajectory=clone_size_traj/total_clone_size_trajectory

        col=np.random.choice(multiple_mutant_colors[mm])
        ax1.plot(np.linspace(0, T*dt, T), clone_frequency_trajectory,linewidth=5, color='w') #plot line   
        ax1.plot(np.linspace(0, T*dt, T), clone_frequency_trajectory,linewidth=3, color=col) #plot line   



    #format plot
    ax1.set_yscale('log')
    ax1.set_xlim(0, T*dt)
    ax1.set_ylim(1*10**-5, 1.2)

    y_major_ticks = [ 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]
    y_major_tick_labels = [ "0.001%", "0.01%", "0.1%", "1%", "10%", "100%"]

    x_major_ticks = [0,10, 20,30, 40,50, 60, 70, 80]
    x_major_tick_labels = [ "0", "10", "20", "30", "40", "50", "60", "70", "80"]

    ax1.set_yticks(y_major_ticks)
    ax1.set_yticklabels(y_major_tick_labels, fontsize = 21)
    ax1.set_xticks(x_major_ticks)
    ax1.set_xticklabels(x_major_tick_labels, fontsize = 21)
    ax1.set_title('Clone sizes, simulation ID: '+str(sim), fontsize = 21)


    ax1.grid(axis = 'y', which = 'both', zorder = 0, linestyle = ':')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.yaxis.set_tick_params(width=1.5, color = grey3, length = 6)
    ax1.xaxis.set_tick_params(width=1.5, color = grey3, length = 6)
    
    plt.show()
    
    
def plot_variant_trajectories(variant_trajectories, T, dt, sim):
    
    plt.close('all')
    fig, ax1 = plt.subplots(1, 1, figsize = (20, 8))
    plt.subplots_adjust(hspace=0.3)

    for variant_id, variant_entries in variant_trajectories.items():
        
        mtraj=variant_entries['mutation_size_trajectory']
        fitness=variant_entries['mutation_fitness']
        mm_number=variant_entries['multiple_mutant_number']

        col=np.random.choice(multiple_mutant_colors[mm_number])
        ax1.plot(np.linspace(0, T*dt, T), mtraj,linewidth=5, color='w') #plot line   
        ax1.plot(np.linspace(0, T*dt, T), mtraj,linewidth=3, color=col) #plot line   


    #format plot
    ax1.set_yscale('log')
    ax1.set_xlim(0, T*dt)
    ax1.set_ylim(1*10**-5, 2)

    y_major_ticks = [ 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]
    y_major_tick_labels = [ "0.001%", "0.01%", "0.1%", "1%", "10%", "100%"]

    x_major_ticks = [0,10, 20,30, 40,50, 60, 70, 80]
    x_major_tick_labels = [ "0", "10", "20", "30", "40", "50", "60", "70", "80"]

    ax1.set_yticks(y_major_ticks)
    ax1.set_yticklabels(y_major_tick_labels, fontsize = 21)
    ax1.set_xticks(x_major_ticks)
    ax1.set_xticklabels(x_major_tick_labels, fontsize = 21)
    ax1.set_title('Variant frequencies, simulation ID: '+str(sim), fontsize = 21)


    ax1.grid(axis = 'y', which = 'both', zorder = 0, linestyle = ':')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.yaxis.set_tick_params(width=1.5, color = grey3, length = 6)
    ax1.xaxis.set_tick_params(width=1.5, color = grey3, length = 6)
    
    plt.show()
    


# Run lots of sims

In [1]:
N=10**5
T=800
dt=0.1
Ub=1*10**-5
number_of_sims=100000000
s1=0.03
s2=s1*2
s3=s1*3
s4=s1*4
simulated_cases={}
simulated_controls={}

dx_ages=[]
case_count=0

j = 0
for sim in range(number_of_sims):
    
    clone_entries={}
    clone_entries['clone_size_trajectory']=np.zeros(T)
    clone_entries['clone_size_trajectory'][0]=N
    clone_entries['mutations']={}
    clone_entries['current_clone_size']=N
    clone_entries['fitness']=0.0
    clone_entries['children']=[]
    clone_entries['parent']=[]
    clones={0: clone_entries}

    latest_mutation_dict={'last_mutation':0}
    

    quandruple_mutant_frequency = 0
    list_of_t1_t2_t3_t4_n4 = [-10,-10,-10,-10,-10]
    mutations_acquired = [False,False,False,False]

    for t in range(T):
    #     print('t=',t)
        mutate_population(clones, latest_mutation_dict, Ub, None, t, dt, T)
        mf, total_pop = mean_fitness(clones)
        select_population(clones, mf, t, dt, total_pop, list_of_t1_t2_t3_t4_n4)
        purge(clones)
        aml_dx, quandruple_mutant_frequency = aml_diagnosis_Poon(clones)
        
        if aml_dx:
            diagnosis_time=t*dt
#             dx_ages.append(diagnosis_time)     
            simulated_cases[sim]={'clones':clones, 'aml_dx':aml_dx, 'diagnosis_time':diagnosis_time}
            case_count += 1
#             print(case_count)
#             if len(simulated_cases.items()) != 0: 
#                 print(case_count, simulated_cases.keys())
            
#             print('AMl diagnosed', case_count, 'quandruple_mutant_frequency:', quandruple_mutant_frequency)

            break
            
        



    if sim % 5000==0:
        print('case count: ', case_count, 'across sim no: ', sim)
        
    path = 'Discrete_s//k_4/mu_s_1times10minus5_3p_6//'
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    barcode_dict_filename='Discrete_s/k_4/mu_s_1times10minus5_3p_6//diagnosis_ages_sim_no'+str(sim)+'.txt'
        
#         with open(barcode_dict_filename, 'w') as outfile:
#             json.dump(dx_ages, outfile)
            
        
    if case_count % 50==0:
        with open('Discrete_s/k_4//mu_s_1times10minus5_3p_6//clonal_architecture_'+str(case_count)+'.pickle', 'wb') as handle:
#             pickle.dump(simulated_cases, handle, protocol=pickle.HIGHEST_PROTOCOL)
            pickle.dump(simulated_cases, handle)
        
#         print(len(simulated_cases.keys()))
        
    if case_count % 50==1:
        
        simulated_cases={}
        simulated_cases[sim]={'clones':clones, 'aml_dx':aml_dx, 'diagnosis_time':diagnosis_time}
        
    if case_count>5001:
        break

        

with open('Discrete_s/k_4//mu_s_1times10minus5_3p_6//clonal_architecture_'+str(case_count)+'.pickle', 'wb') as handle:
    pickle.dump(simulated_cases, handle)
    
print('fraction of cases by 80 = ', case_count/number_of_sims)    
